# Semeval 2025 Task 10

## Subtask 1: Entity Framing -- Multilingual

Given a news article and a list of mentions of named entities (NEs) in the article, assign for each such mention one or more roles using a predefined taxonomy of fine-grained roles covering three main type of roles: protagonists, antagonists, and innocent. This is a multi-label multi-class text-span classification task.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
import os

In [2]:
root_dir = "../"

In [3]:
data = []
ignore_folders = ['.DS_Store']

base_dir_documents = root_dir + '../data/semeval_data/train/raw-documents'

for language_folder in os.listdir(base_dir_documents):

    if language_folder in ignore_folders:
        continue

    language_path = os.path.join(base_dir_documents, language_folder)
    if os.path.isdir(language_path):
        for root, _, files in os.walk(language_path):
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)

                    article_id = file
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()

                    data.append({
                        'language': language_folder,
                        'article_id': article_id,
                        'content': content
                    })

documents_df = pd.DataFrame(data)

In [4]:
documents_df.shape

(1709, 3)

In [5]:
documents_df.sample(5)

,language,article_id,content
952,HI,HI_68.txt,यूक्रेन में रूसी सेना द्वारा किए गए सबसे बड़े ...
1371,EN,EN_CC_100136.txt,Climate Science Denial Group GWPF Admits It Us...
567,BG,BG_546.txt,The National Interest: Удари по ВСУ с бомби FA...
1135,HI,HI_197.txt,"बर्बरता पर उतर आई रूसी सेना, यूक्रेनियों के खू..."
1610,EN,EN_UA_100840.txt,Warning US could be ‘hit with most debilitatin...


In [6]:
documents_df['article_id'].unique

<bound method Series.unique of 0        RU-URW-1161.txt
1        RU-URW-1175.txt
2        RU-URW-1149.txt
3        RU-URW-1015.txt
4        RU-URW-1001.txt
              ...       
1704    EN_UA_008072.txt
1705    EN_CC_300151.txt
1706    EN_CC_200145.txt
1707    EN_UA_015962.txt
1708    EN_CC_300179.txt
Name: article_id, Length: 1709, dtype: object>

In [7]:
base_dir_labels = root_dir + '../data/semeval_data/train/labels'

raw_annotation_data = []

for language_folder in os.listdir(base_dir_labels):

    if language_folder in ignore_folders:
        continue

    language_path = os.path.join(base_dir_labels, language_folder)
    if os.path.isdir(language_path):
        for root, _, files in os.walk(language_path):
            label_file = 'subtask-1-annotations.txt'
            file_path = os.path.join(root, label_file)
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    parts = line.strip().split('\t')
                    article_id = parts[0]
                    entity_mention = parts[1]
                    start_offset = int(parts[2])
                    end_offset = int(parts[3])
                    main_role = parts[4]

                    sub_roles = parts[5:]
                    raw_annotation_data.append({
                        "article_id": article_id,
                        "entity_mention": entity_mention,
                        "start_offset": start_offset,
                        "end_offset": end_offset,
                        "main_role": main_role,
                        "sub_roles": sub_roles,
                    })

annotations_df = pd.DataFrame(raw_annotation_data)

In [8]:
annotations_df.head()

,article_id,entity_mention,start_offset,end_offset,main_role,sub_roles
0,RU-URW-1080.txt,Ермак,155,159,Antagonist,[Incompetent]
1,RU-URW-1080.txt,Трамп,492,496,Protagonist,[Peacemaker]
2,RU-URW-1013.txt,Украины,108,114,Innocent,[Victim]
3,RU-URW-1145.txt,Российские войска,105,121,Protagonist,[Guardian]
4,RU-URW-1145.txt,ВСУ,131,133,Antagonist,[Terrorist]


In [9]:
annotations_df.shape

(5262, 6)

In [10]:
from collections import defaultdict

main_to_sub = defaultdict(set)

for record in raw_annotation_data:
    main_role = record['main_role']
    sub_roles = record['sub_roles']
    for main, sub in zip([main_role], sub_roles):
        main_to_sub[main].add(sub)

main_to_sub = {main: list(subs) for main, subs in main_to_sub.items()}

In [11]:
main_to_sub

{'Antagonist': ['Corrupt',
  'Instigator',
  'Spy',
  'Terrorist',
  'Incompetent',
  'Saboteur',
  'Deceiver',
  'Tyrant',
  'Foreign Adversary',
  'Conspirator',
  'Bigot',
  'Traitor'],
 'Protagonist': ['Underdog',
  'Peacemaker',
  'Martyr',
  'Virtuous',
  'Rebel',
  'Guardian'],
 'Innocent': ['Forgotten', 'Exploited', 'Scapegoat', 'Victim']}

In [12]:
dataset = pd.merge(documents_df, annotations_df, on='article_id')
dataset.head()

,language,article_id,content,entity_mention,start_offset,end_offset,main_role,sub_roles
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,НАТО,173,176,Antagonist,[Foreign Adversary]
1,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Украине,262,268,Innocent,[Exploited]
2,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Вадим Колесниченко,414,431,Protagonist,[Virtuous]
3,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,России,1292,1297,Innocent,[Victim]
4,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,Макрона,3422,3428,Antagonist,[Incompetent]


In [13]:
print(dataset.iloc[6].content)

Возможность признания Аллы Пугачевой иностранным агентом

"В свете недавних событий вокруг Аллы Пугачевой возник вопрос о возможности признания её иностранным агентом. Этот сложный юридический процесс требует тщательного рассмотрения ряда факторов. Управляющий партнер компании ""Русяев и партнеры"", юрист Илья Русяев, пояснил, что для признания Пугачевой иноагентом необходимо установить наличие иностранного финансирования или влияния. Учитывая её международную известность и связи, можно предположить наличие зарубежных источников дохода. Однако этого недостаточно нужно доказать, что эти средства используются для политической деятельности в России.

Ключевым аспектом в данном случае может стать публичная позиция Пугачевой. Её недавний пост о ситуации в Киеве может быть расценен как распространение информации, дискредитирующей действия российских Вооруженных сил, что может интерпретироваться как политическая деятельность. Для инициирования процесса Министерству юстиции потребуется собрать

In [14]:
print(dataset.iloc[6].entity_mention)

Министерству юстиции


In [15]:
dataset.head()

,language,article_id,content,entity_mention,start_offset,end_offset,main_role,sub_roles
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,НАТО,173,176,Antagonist,[Foreign Adversary]
1,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Украине,262,268,Innocent,[Exploited]
2,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Вадим Колесниченко,414,431,Protagonist,[Virtuous]
3,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,России,1292,1297,Innocent,[Victim]
4,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,Макрона,3422,3428,Antagonist,[Incompetent]


In [16]:
dataset.shape

(5262, 8)

In [17]:
dataset['main_role'].explode().value_counts()

main_role
Antagonist     2435
Protagonist    1759
Innocent       1068
Name: count, dtype: int64

In [18]:
dataset['sub_roles'].explode().value_counts()

sub_roles
Victim               939
Guardian             808
Foreign Adversary    781
Virtuous             493
Instigator           376
Peacemaker           294
Incompetent          293
Tyrant               279
Conspirator          226
Rebel                224
Deceiver             209
Terrorist            201
Underdog             188
Corrupt              157
Exploited            104
Saboteur              82
Bigot                 69
Traitor               49
Scapegoat             33
Forgotten             32
Martyr                31
Spy                   20
Name: count, dtype: int64

In [19]:
dataset.isnull().sum()

language          0
article_id        0
content           0
entity_mention    0
start_offset      0
end_offset        0
main_role         0
sub_roles         0
dtype: int64

## Setting up

In [20]:
def get_context(row, window=50, char_window=500):
    content = row['content']
    start = int(row['start_offset'])
    end = int(row['end_offset'])

    context_start = max(0, start - char_window)
    context_end = min(len(content), end + char_window)

    context_before = content[context_start:start]
    context_after = content[end:context_end]

    # Ensure we don't cut words in half
    context_before = ' '.join(context_before.split()[-window:])
    context_after = ' '.join(context_after.split()[:window])

    return context_before, context_after

dataset['context_before'], dataset['context_after'] = zip(*dataset.apply(get_context, axis=1))

In [21]:
example_context = "Moscow: Russia will ask the UN Security Council for an investigation"
entity_mention = "Russia"
start_offset = 8
end_offset = 13

example_df = pd.DataFrame({
    'content': [example_context],
    'entity_mention': [entity_mention],
    'start_offset': [start_offset],
    'end_offset': [end_offset]
})

example_df

,content,entity_mention,start_offset,end_offset
0,Moscow: Russia will ask the UN Security Counci...,Russia,8,13


In [22]:
get_context(example_df.iloc[0])

('Moscow:', 'a will ask the UN Security Council for an investigation')

In [23]:
dataset['entity_context'] = dataset['context_before'] \
                            + " " + dataset['entity_mention'] \
                            + " " + dataset['context_after']

In [24]:
dataset.drop(columns=['context_before', 'context_after'], inplace=True)
dataset.head()

,language,article_id,content,entity_mention,start_offset,end_offset,main_role,sub_roles,entity_context
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,НАТО,173,176,Antagonist,[Foreign Adversary],В ближайшие два месяца США будут стремиться к ...
1,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Украине,262,268,Innocent,[Exploited],В ближайшие два месяца США будут стремиться к ...
2,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Вадим Колесниченко,414,431,Protagonist,[Virtuous],стремиться к эскалации конфликта на Украине – ...
3,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,России,1292,1297,Innocent,[Victim],"долг вырос более чем на 7 триллионов долларов""..."
4,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,Макрона,3422,3428,Antagonist,[Incompetent],"институтами ЕС. Оппозиция """"Национальное объед..."


In [25]:
dataset.head()

,language,article_id,content,entity_mention,start_offset,end_offset,main_role,sub_roles,entity_context
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,НАТО,173,176,Antagonist,[Foreign Adversary],В ближайшие два месяца США будут стремиться к ...
1,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Украине,262,268,Innocent,[Exploited],В ближайшие два месяца США будут стремиться к ...
2,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Вадим Колесниченко,414,431,Protagonist,[Virtuous],стремиться к эскалации конфликта на Украине – ...
3,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,России,1292,1297,Innocent,[Victim],"долг вырос более чем на 7 триллионов долларов""..."
4,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,Макрона,3422,3428,Antagonist,[Incompetent],"институтами ЕС. Оппозиция """"Национальное объед..."


## Cleaning data

In [26]:
!python3 -m spacy download xx_ent_wiki_sm
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 7.6 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 5.6 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [27]:
!python3 -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [28]:
language_model_map = {
    "BG": "xx_ent_wiki_sm",
    "EN": "en_core_web_sm",
    "HI": "xx_ent_wiki_sm",
    "PT": "pt_core_news_sm",
    "RU": "ru_core_news_sm",
}

In [29]:
!pip3 install emoji

In [30]:
import spacy
import re
import emoji

nlp_models = {lang: spacy.load(model) for lang, model in language_model_map.items()}

def clean_article(article_text, language_code, entity_mention):
    nlp = nlp_models.get(language_code, nlp_models["EN"])

    entity_mention = entity_mention.strip()
    entity_parts = entity_mention.lower().split()

    article_text = article_text.replace('"', '"').replace('"', '"').replace("'", "'")
    article_text = re.sub(r'\s+', ' ', article_text)
    article_text = re.sub(r'\s*([.,!?])\s*', r'\1 ', article_text)
    article_text = re.sub(
        r'(http\S+|www\S+|https\S+|[a-zA-Z0-9.-]+\.com|'
        r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+|@[A-Za-z0-9_]+)',
        '',
        article_text
    )

    coref_markers = ["he", "she", "they", "it", "their", "his", "her"]
    doc = nlp(article_text)

    cleaned_tokens = []
    i = 0
    while i < len(doc):
        token = doc[i]

        if token.is_space or emoji.is_emoji(token.text):
            i += 1
            continue

        if i + len(entity_parts) <= len(doc):
            potential_entity = ' '.join(t.text.lower() for t in doc[i:i + len(entity_parts)])
            if potential_entity == ' '.join(entity_parts):
                original_entity = ' '.join(t.text for t in doc[i:i + len(entity_parts)])
                cleaned_tokens.append(f"[ES] {original_entity} [EE] ")
                i += len(entity_parts)
                continue

        if token.text.lower() in entity_parts:
            cleaned_tokens.append(f"[ES] {token.text} [EE] ")
        elif token.text.lower() in coref_markers:
            cleaned_tokens.append(f"[COREF] {token.text}")
        elif token.ent_type_ in ["PERSON", "ORG", "GPE"]:
            cleaned_tokens.append(token.text)
        else:
            cleaned_tokens.append(token.text.lower())

        if token.whitespace_:
            cleaned_tokens.append(" ")

        i += 1

    return "".join(cleaned_tokens).strip()

dataset["entity_context"] = dataset.apply(
    lambda row: clean_article(row["entity_context"], row["language"], row["entity_mention"]),
    axis=1
)

In [31]:
print(dataset.iloc[6].entity_context)

недостаточно нужно доказать, что эти средства используются для политической деятельности в россии. ключевым аспектом в данном случае может стать публичная позиция пугачевой. её недавний пост о ситуации в киеве может быть расценен как распространение информации, дискредитирующей действия российских Вооруженных сил, что может интерпретироваться как политическая деятельность. для инициирования процесса мини [ES] Министерству юстиции [EE] требуется собрать доказательную базу, включая анализ её финансовых операций, публичных выступлений и постов в социальных сетях. сам факт критических высказываний недостаточен необходимо доказать систематический характер такой деятельности и её связь с иностранным влиянием. если Минюст решит включить пугачеву в реестр иноагентов, она получит уведомление и право обжаловать это решение


In [32]:
from sklearn.preprocessing import LabelEncoder

le_main_role = LabelEncoder()
dataset['main_role_encoded'] = le_main_role.fit_transform(dataset['main_role'])

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_sub_role = MultiLabelBinarizer()
dataset.loc[:, 'sub_roles_encoded'] = list(mlb_sub_role.fit_transform(dataset['sub_roles']))

print(dataset.shape)
dataset.head()

(5262, 11)


,language,article_id,content,entity_mention,start_offset,end_offset,main_role,sub_roles,entity_context,main_role_encoded,sub_roles_encoded
0,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,НАТО,173,176,Antagonist,[Foreign Adversary],в ближайшие два месяца сша будут стремиться к ...,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Украине,262,268,Innocent,[Exploited],в ближайшие два месяца сша будут стремиться к ...,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,Вадим Колесниченко,414,431,Protagonist,[Virtuous],стремиться к эскалации конфликта на украине – ...,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,RU,RU-URW-1161.txt,В ближайшие два месяца США будут стремиться к ...,России,1292,1297,Innocent,[Victim],"долг вырос более чем на 7 триллионов долларов""...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,RU,RU-URW-1175.txt,В ЕС испугались последствий популярности правы...,Макрона,3422,3428,Antagonist,[Incompetent],"институтами ес. оппозиция """"Национальное объед...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [34]:
base_save_folder_dir = './saved/'

os.makedirs(base_save_folder_dir, exist_ok=True)

In [35]:
datasets_folder = os.path.join(base_save_folder_dir, 'Datasets')
label_encoder_folder = os.path.join(base_save_folder_dir, 'LabelEncoders')
misc_folder = os.path.join(base_save_folder_dir, 'Misc')

In [36]:
import pickle

with open(os.path.join(datasets_folder, 'dataset_cleaned.pkl'), 'wb') as f:
    pickle.dump(dataset, f)

with open(os.path.join(label_encoder_folder, 'le_main_role.pkl'), 'wb') as f:
    pickle.dump(le_main_role, f)

with open(os.path.join(label_encoder_folder, 'mlb_sub_role.pkl'), 'wb') as f:
    pickle.dump(mlb_sub_role, f)

with open(os.path.join(misc_folder, 'main_to_sub.pkl'), 'wb') as f:
    pickle.dump(main_to_sub, f)